In [3]:
import pyads
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

plc_buffer_name = 'Main.profiler_buffer'
plc_numlayers_name = 'Main.FB_NN.nn.num_layers'
plc_numneurons_name = 'Main.FB_NN.nn.layers[[index]].num_neurons'
plc_MeasureFinished_name = 'Main.MeasureFinished'

num_layers = [6, 5, 4, 3]
num_neurons = [500, 400, 300, 200, 100, 10]

In [7]:



measurements = []
with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
    for i in num_layers:
        plc.write_by_name(plc_numlayers_name,i,pyads.PLCTYPE_UINT)
        for j in num_neurons:
            for k in range(i-2):
                plc.write_by_name(plc_numneurons_name.replace('[index]',f'{k+1}'),j,pyads.PLCTYPE_UINT)
            plc.write_by_name(plc_MeasureFinished_name, False ,pyads.PLCTYPE_BOOL)
            while(1):
                plc_MeasureFinished = plc.read_by_name(plc_MeasureFinished_name, pyads.PLCTYPE_BOOL)
                if plc_MeasureFinished:
                    profiler_buffer = plc.read_by_name(plc_buffer_name, pyads.PLCTYPE_DWORD*1000)
                    print(f'num_layers:{i}, num_neurouns:{j}\n')
                    print(profiler_buffer[0:10])
                    for k in profiler_buffer:
                        measurements.append([i,j,k])
                    break

df = pd.DataFrame(measurements, columns=['num_layers', 'num_neurons', 'measurement'])

num_layers:6, num_neurouns:500

[2841, 2803, 2801, 2808, 2855, 2777, 2822, 2799, 2786, 2807]
num_layers:6, num_neurouns:400

[1781, 1784, 1779, 1782, 1770, 1807, 1771, 1788, 1794, 1800]
num_layers:6, num_neurouns:300

[1051, 1021, 1033, 1025, 1015, 1033, 1003, 992, 1011, 1017]
num_layers:6, num_neurouns:200

[454, 453, 452, 458, 456, 453, 455, 453, 452, 452]
num_layers:6, num_neurouns:100

[122, 122, 122, 122, 121, 122, 123, 122, 122, 122]
num_layers:6, num_neurouns:10

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
num_layers:5, num_neurouns:500

[1965, 1859, 1901, 1866, 1895, 1897, 1879, 1852, 1863, 1852]
num_layers:5, num_neurouns:400

[1235, 1232, 1241, 1218, 1222, 1224, 1206, 1266, 1265, 1245]
num_layers:5, num_neurouns:300

[683, 702, 706, 697, 691, 696, 692, 698, 709, 683]
num_layers:5, num_neurouns:200

[330, 311, 311, 313, 311, 312, 311, 311, 311, 311]
num_layers:5, num_neurouns:100

[86, 86, 86, 87, 86, 86, 86, 86, 86, 86]
num_layers:5, num_neurouns:10

[3, 2, 3, 3, 2, 3, 2, 3, 3, 2]
num_lay

In [8]:
df.to_pickle('exe_time_measurement_sigmoid_arch.pkl')

In [9]:
color_map = {6: 'rgb(255,0,0)', 5: 'rgb(0,255,0)', 4: 'rgb(0,0,255)', 3: 'rgb(255,255,0)'}

fig = px.box(df, x="num_neurons", y="measurement", color="num_layers",boxmode = 'overlay', color_discrete_map=color_map)
fig.update_layout(xaxis_title="Number of Neurons in each layer", yaxis_title="Execution Time [us]", legend_title = 'Number of Layers')
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = num_neurons
    )
)

for i in num_layers:
    mean = []
    for j in num_neurons:
        mean.append(df.loc[(df['num_layers'] == i) & (df['num_neurons'] == j)]['measurement'].mean())
    fig.add_trace(go.Scatter(x=num_neurons, y=mean, showlegend=False,  marker_color=color_map[i] ))
fig.show()